In [1]:
import csv
import numpy as np
import os
import pandas as pd

from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from sklearn.preprocessing import normalize
from sklearn import preprocessing
import torch
import matplotlib.pyplot as plt
import optuna
from optuna.trial import TrialState


In [2]:
import torch.nn as nn
import torch.nn.functional as F
from tqdm.notebook import tqdm
import torch.optim as optim

In [3]:
batch_size = 256
EPOCHS=10

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)

cuda:0


In [5]:
fname="/gpfs/work/sharmas/mc-snow-data/Timeseries_data.csv"
df=pd.read_csv(fname)

In [6]:
def dataset_transform(dataset):
   #def norm(x):
   #    return (x-np.mean(x))/(np.std(x))
   #
   #for i in range (norm_start,dataset.shape[-1]):
   #    print (i)
   #    dataset[:,i]=norm(dataset[:,i])
        
    
    data = torch.FloatTensor(dataset).to(device)
    train_split = int(len(data) * 0.8)
    test_split = len(data) - train_split
    test_dataset=data[train_split:,:]
    train_dataset=data [0:train_split, :]


    train_split=int(len(train_dataset)*0.8)
    val_split=len(train_dataset)-train_split
    train_dataset, val_dataset=random_split(train_dataset, [train_split, val_split])

    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
    val_dataloader = DataLoader(val_dataset, shuffle=False, batch_size=batch_size)
    test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)  
    
    
    return train_dataloader,val_dataloader,test_dataloader,train_dataset,val_dataset, test_dataset

In [7]:
def create_dataset (d, time):
    inputs=[]
    outputs=[]
    timestep=[]
    for i in range (len(d)-1):
        #print (i)
        if time[i]< time[i+1]:
            data_in=d[i]
            data_out=d[i+1]
            inputs.append(data_in)
            outputs.append(data_out)
            timestep.append(time[i])
    dataset=np.concatenate(( (np.asarray(timestep)).reshape(len(timestep),1),np.asarray(inputs),np.asarray(outputs)),axis=1)
    return dataset
    

In [8]:
arr_df=(df.to_numpy())
time=arr_df[:,0]
d=arr_df[:,1:6]
d=np.delete(d,2,1)
means, sds = np.full(4, np.nan), np.full(4, np.nan)
for i in range(d.shape[1]):
    means[i], sds[i] = np.mean(d[:, i]), np.std(d[:, i])
    d[:, i] = (d[:, i] - means[i]) / sds[i]
dataset=create_dataset(d, time)


# For 3 inputs

In [7]:
def create_dataset (d, time,lwo,r):
    inputs=[]
    outputs=[]
    timestep=[]
    lo=[]
    lr=[]
    for i in range (len(d)-1):
        #print (i)
        if time[i]< time[i+1]:
            data_in=d[i]
            data_out=d[i+1]
            inputs.append(data_in)
            outputs.append(data_out)
            timestep.append(time[i])
            lo.append(lwo[i])
            lr.append(r[i])
    dataset=np.concatenate((np.asarray(inputs),np.asarray(outputs)),axis=1)
    return np.asarray(timestep),np.asarray(lo),np.asarray(lr),dataset
    

In [8]:
arr_df=(df.to_numpy())
time=arr_df[:,0]
d=arr_df[:,1:6]
d=np.delete(d,[2,3],1)#Forecast only Lc, not Lr

means, sds = np.full(3, np.nan), np.full(3, np.nan)
for i in range(d.shape[1]):
    means[i], sds[i] = np.mean(d[:, i]), np.std(d[:, i])
    d[:, i] = (d[:, i] - means[i]) / sds[i]

timestep,lo,lr,dataset=create_dataset(d, time,arr_df[:,-4],arr_df[:,4])
dataset=np.concatenate ((timestep.reshape(478366,1),lo.reshape(478366,1),lr.reshape(478366,1), dataset), axis=1)

In [13]:
def define_model(trial):
    n_layers = trial.suggest_int("n_layers", 2, 6)
    activation_func=trial.suggest_categorical('activation',[nn.Tanh,nn.LeakyReLU,nn.ReLU,nn.ELU,nn.Hardtanh,nn.PReLU,nn.ReLU6,nn.RReLU,nn.SiLU])
    layers = []

    in_features = 3
    for i in range(n_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i), 4, 128)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(activation_func())
        p = trial.suggest_float("dropout_l{}".format(i), 0.2, 0.5)
        layers.append(nn.Dropout(p))

        in_features = out_features
    layers.append(nn.Linear(in_features, 3))
    return nn.Sequential(*layers)

In [10]:
def calc_mse(y,y_pred):

    return np.mean((y - y_pred)**2)

In [11]:
criterion=nn.MSELoss()

In [12]:
# Get the dataset.
train_dataloader,val_dataloader,test_dataloader,train_dataset,val_dataset, test_dataset = dataset_transform(dataset)


In [14]:
def objective(trial):

    model = define_model(trial).to(device)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=1e-3)

    # Training 
    for epoch in range(EPOCHS):
        print(f'epoch {epoch}')
        model.train()
        for i, data in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):

            inputs=data[:,3:6].to(device)
            outputs=data[:,6:].to(device)

            optimizer.zero_grad()
            preds = model(inputs)
            loss = criterion(outputs, preds)
            loss.backward()
            optimizer.step()

        # Validation of the model.
        model.eval()
        predictions=[]
        targets=[]
        with torch.no_grad():
            d = val_dataset[:]

            inputs = d[:,3:6]
            targets = d[:,6:]

            predictions = model(inputs)

            inputs, predictions, targets = inputs.to('cpu').numpy(), predictions.to('cpu').numpy(), targets.to('cpu').numpy()
  
                  
        mse_val = calc_mse(np.asarray(targets),np.asarray(predictions))

        trial.report(mse_val, epoch)
        print ("MSE Validation:",mse_val)
        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return mse_val

In [15]:
if __name__ == "__main__":
    study = optuna.create_study()
    study.optimize(objective, n_trials=100, timeout=600)

    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2021-03-26 08:03:21,631] A new study created in memory with name: no-name-494309c9-4475-4030-a2a9-8629f3d5c46f


epoch 0


/gpfs/home/sharmas/miniconda3/lib/python3.8/site-packages/optuna/distributions.py:426: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <class 'torch.nn.modules.activation.Tanh'> which is of type type.
  warnings.warn(message)
/gpfs/home/sharmas/miniconda3/lib/python3.8/site-packages/optuna/distributions.py:426: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <class 'torch.nn.modules.activation.LeakyReLU'> which is of type type.
  warnings.warn(message)
/gpfs/home/sharmas/miniconda3/lib/python3.8/site-packages/optuna/distributions.py:426: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <class 'torch.nn.modules.activation.ReLU'> which is of type type.
  warnings.warn(message)
/gpfs/home/sharmas/miniconda3/lib/python3

  0%|          | 0/1196 [00:00<?, ?it/s]

MSE Validation: 0.774459
epoch 1


...

  0%|          | 0/1196 [00:00<?, ?it/s]

MSE Validation: 0.006054841
epoch 9


  0%|          | 0/1196 [00:00<?, ?it/s]

[I 2021-03-26 08:13:24,099] Trial 19 finished with value: 0.0044861771166324615 and parameters: {'n_layers': 2, 'activation': <class 'torch.nn.modules.activation.PReLU'>, 'n_units_l0': 83, 'dropout_l0': 0.2608790636048178, 'n_units_l1': 40, 'dropout_l1': 0.20224067253365238, 'optimizer': 'RMSprop'}. Best is trial 12 with value: 0.0023704629857093096.


MSE Validation: 0.004486177
Study statistics: 
  Number of finished trials:  20
  Number of pruned trials:  7
  Number of complete trials:  13
Best trial:
  Value:  0.0023704629857093096
  Params: 
    n_layers: 2
    activation: <class 'torch.nn.modules.activation.LeakyReLU'>
    n_units_l0: 104
    dropout_l0: 0.21535053659219258
    n_units_l1: 75
    dropout_l1: 0.20923036991491112
    optimizer: RMSprop
